In [1]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.models import load_model
import pickle

In [2]:
##Loading the ANN trained model:
model = load_model("model.h5")

##Loading the scaler pickle file:
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

##Loading the encoder pickle files:
with open ('le_gender.pkl', 'rb') as file:
    le_gender = pickle.load(file)

with open ('ohe_geo.pkl', 'rb') as file:
    ohe_geo = pickle.load(file)


In [79]:
#Example input data
input_data = {
    "CreditScore" : 60,
    "Geography" : "France",
    "Gender" : "Male",
    "Age" : 40,
    "Tenure" : 3,
    "Balance" : 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000,
}

In [61]:
geo_ohencoded = ohe_geo.transform([[input_data["Geography"]]]).toarray()
ohe_geo_columns = ohe_geo.get_feature_names_out(["Geography"])


/Users/sls/Documents/GitHub/DL_projects/ANNchrun/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [66]:
geo_ohencoded_df = pd.DataFrame(geo_ohencoded, columns = ohe_geo_columns)
geo_ohencoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [67]:
type(geo_ohencoded_df), type(input_data)

(pandas.core.frame.DataFrame, dict)

In [80]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,60,France,Male,40,3,60000,2,1,1,50000


In [71]:
type(geo_ohencoded_df), type(input_df)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [81]:
#Encoding categorical feature
input_df["Gender"] = le_gender.transform(input_df["Gender"])

In [83]:
#creating backup of a original input
original_input_data = input_data.copy()
original_input_data

{'CreditScore': 60,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [87]:
#Concatinating the geo_ohencoded_df to input_data
input_data = pd.concat([input_df.drop("Geography", axis=1),geo_ohencoded_df], axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,60,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [88]:
#Sacling the input data
scaled_input = scaler.transform(input_data)
scaled_input

array([[-6.13995993,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [89]:
#Predicting churn
prediction = model.predict(scaled_input)
prediction

1/1 [==============================] - 0s 109ms/step


array([[0.01499166]], dtype=float32)

In [90]:
prediction_prob = prediction[0][0]
prediction_prob

0.014991663

In [95]:
def churn(x):
    if x > 0.5:
        print("The customer is ikely to CHURN")
    else:
        print("The customer is ikely NOT to churn")

In [97]:
churn(prediction_prob)

The customer is ikely NOT to churn
